# Crime classification in San Francisco

In [2]:
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [10]:
train_data= pd.read_csv('data/pre_processing_train_data.csv')
train_data=train_data.iloc[:,1:]

In [13]:
train_sample = train_data.sample(n=10000)
#dfsample = rebuild_data(df,dfsample) # new line added in order to imporve the sample
train_labels=train_sample['Category']
print(len(train_labels.unique()))
train_labels.head()
train_sample.drop('Category',inplace =True, axis=1)
train_sample.head()

37


,X,Y,Year,X_Bar_District,Y_Bar_District,Sin_Year,Cos_Year,Sin_Hour,Cos_Hour,Sin_Day_m,Cos_Day_m,Sin_Month,Cos_Month,Sin_Day_w,Cos_Day_w
421375,-122.420926,37.762182,0.000000,-122.419409,37.759961,-0.500000,-0.866025,-0.500000,-0.866025,0.968077,-0.250653,1.224647e-16,-1.000000e+00,0.781831,0.623490
280407,-122.410769,37.783215,0.333333,-122.412503,37.783802,0.725374,-0.688355,0.866025,-0.500000,-0.101168,-0.994869,-5.000000e-01,-8.660254e-01,-0.974928,-0.222521
830654,-122.416829,37.788179,-1.000000,-122.409619,37.795685,0.358368,0.933580,0.258819,0.965926,-0.897805,-0.440394,-8.660254e-01,-5.000000e-01,0.433884,-0.900969
87145,-122.412931,37.783834,0.833333,-122.412503,37.783802,0.548293,0.836286,0.500000,0.866025,-0.651372,-0.758758,1.000000e+00,6.123234e-17,0.974928,-0.222521
404783,-122.412597,37.783932,0.000000,-122.412503,37.783802,-0.426569,-0.904455,-0.258819,-0.965926,0.724793,0.688967,-1.000000e+00,-1.836970e-16,-0.433884,-0.900969


In [14]:
# Load Library
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import log_loss
#from sklearn.neural_network import MLPClassifier
#from sklearn.neighbors import KNeighborsClassifier

#X=(dfsample-dfsample.mean())/dfsample.std()
X=train_sample
y=train_labels

# Split the training test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#  Fit a Random Forest model
clf = RandomForestClassifier(n_estimators=150,max_depth=None,min_samples_leaf=20,min_samples_split=40,max_features='sqrt',bootstrap=False,random_state=42)
#clf = MLPClassifier(hidden_layer_sizes=(100),solver='sgd',learning_rate='invscaling',random_state=1)
#clf=KNeighborsClassifier(n_neighbors=1000)
clf.fit(X_train, y_train)
p_pred = clf.predict_proba(X_test)
print(p_pred.shape)
print(clf)
print(clf.feature_importances_)
print(log_loss(y_test,p_pred))
print(accuracy_score(y_test, np.argmax(p_pred,axis=1)))


(2000, 37)
RandomForestClassifier(bootstrap=False, max_features='sqrt',
                       min_samples_leaf=20, min_samples_split=40,
                       n_estimators=150, random_state=42)
[0.14678676 0.17622443 0.07556345 0.05266752 0.06841475 0.08413312
 0.08353401 0.03550483 0.03868703 0.05756865 0.05002278 0.03649427
 0.03242309 0.03722593 0.02474938]


ValueError: y_true and y_pred contain different number of classes 33, 37. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 29 30 31 35 37]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=150,max_depth=None,min_samples_leaf=20,min_samples_split=40,max_features='sqrt',bootstrap=False,random_state=42)

clf.fit(train_sample, train_labels)

In [ ]:
import joblib
filename = 'rdf_clf.joblib.pkl'
_=joblib.dump(clf, filename, compress=9)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

clf = RandomForestClassifier()
# Number of trees in random forest
n_estimators =[50,100,150]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [None,5,10,15]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [20,50,100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [10,20,50]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 50, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)




    


In [ ]:
print(rf_random.best_params_)
p_pred = rf_random.predict_proba(X_test)
print(p_pred.shape)

print(log_loss(p_pred,y_test))
print(accuracy_score(y_test, np.argmax(p_pred,axis=1)))


In [ ]:
def cross_val_rdf(X,y,nb_range=[100],mss_range=[10]):
    n=len(nb_range)
    m=len(mss_range)
    res=np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            print((nb_range[i],mss_range[j]))
            clf=RandomForestClassifier(n_estimators=nb_range[i],min_samples_split=mss_range[j],min_samples_leaf=mss_range[j]//2,)random_state=42)
            scores=cross_val_score(clf,X,y)
            score=np.mean(scores)
            print(score)
            res[i,j]=score
    (imax,jmax)=np.unravel_index(res.argmax(), res.shape)
    return (nb_range[imax],mss_range[jmax])

cross_val_rdf(X,y,nb_range=[100,150,200],mss_range=[40,50,60])
